# What We Want
We want to manipulate the JSON files and make them so the values for heatmap and move paths show the % difference compared to other players.

In [87]:
import pandas as pd
import json
from pandas import compat
from pprint import pprint

## Proof of concept - Grab all the raw moves data from one player

In [88]:
with open('../../app/data/json/as_color/Adams.asblack_stats.json') as data_file:    
    data = json.load(data_file)

In [61]:
moves = data['moves']

In [66]:
moves_df = pd.DataFrame(moves)

In [77]:
moves_df['Bc1'].sum()

4555.0

In [ ]:
moves_df['Bc1'].sum()

In [73]:
moves_percentage = (moves_df/moves_df.sum())*100

### Save as json, dropping Nan values

In [99]:
moves_percentage.to_json('./test_move_percentage.json')

In [93]:
moves_percentage.apply(lambda x: [x.dropna()], axis=1).to_json('./test_move_percentage.json')

# Load in every player's data to get total average

In [168]:
all_gms = [#'Adams',
    'Akobian','Akopian','Alburt','Alekhine','Alekseev','Almasi','Anand','Anderssen','Andersson','Andreikin','Aronian','Ashley',
'Averbakh','Azmaiparashvili','Bacrot','Bareev','BecerraRivero','Beliavsky','Benjamin','Benko',
           #'Berliner',
           'Bernstein','Bird','Bisguier','Blackburne',
'Blatny','Bogoljubow','Boleslavsky','Bologan','Botvinnik','Breyer','Bronstein','Browne','Bruzon','Bu','Byrne','Capablanca','Carlsen','Caruana',
'Chiburdanidze','Chigorin','Christiansen','DeFirmian',
           #'DeLaBourdonnais',
           'Denker','Ding','DominguezPerez','Dreev','Dzindzichashvili','Ehlvest',
'Eljanov','Euwe','Evans','Fedorowicz','Fine','Finegold','Fischer','Fishbein','Flohr','Gaprindashvili','Gashimov','Gelfand','Geller','Georgiev','Giri',
'Gligoric','Goldin','GrandaZuniga','Grischuk','Gulko','Gunsberg','GurevichD','GurevichM','Harikrishna','Hort','Horwitz','Hou','Huebner','Ibragimov',
'IllescasCordoba','Inarkiev','Ivanchuk','IvanovA','IvanovI','Ivkov','Jakovenko','Janowski','Jobava','Jussupow','Kaidanov','Kamsky','Karjakin',
'Karpov','Kasimdzhanov','Kasparov','Kavalek','Keres','Khalifman','Kholmov','Koneru','Korchnoi','Korobov','Kosteniuk','Kotov','Kramnik',
'Krasenkow','Krush','Kudrin','Lahno','Larsen','Lasker','Lautier','Le','Leko','Levenfish','Li','Lilienthal','Ljubojevic','Lputian','MacKenzie','Malakhov',
'Mamedyarov','Maroczy','Marshall',
           #'McDonnell',
           'McShane','Mecking','Mikenas','Miles','Milov','Morozevich','Morphy','Motylev','Movsesian',
'Muzychuk','Najdorf','Najer','Nakamura','Navara','Negi','Nepomniachtchi','Ni','Nielsen','Nikolic','Nimzowitsch','Nisipeanu','Novikov','Nunn',
'Olafsson','Oll','Onischuk','Pachman','Paehtz','Panno','Paulsen','Petrosian',
#'Philidor',
           'Pillsbury','Pilnik','PolgarJ','PolgarS','PolgarZ','Polugaevsky',
'Ponomariov','Portisch','Psakhis','Quinteros','Radjabov','Reshevsky','Reti','Ribli','Rohde','Rubinstein','Rublevsky','Saemisch','Sakaev','Salov',
'Sasikiran','Schlechter','Seirawan','Serper','Shabalov','Shamkovich','Shirov','Short','Shulman','Smirin','Smyslov','So','Sokolov','Soltis','Spassky',
'Speelman','Spielmann','Stahlberg','Staunton','Stefanova','Stein','Steinitz','Suetin','SultanKhan','Sutovsky','Svidler','Szabo','Taimanov','Tal',
'Tarrasch','Tartakower','Teichmann','Timman','Tiviakov','Tkachiev','Tomashevsky','Topalov','TorreRepetto','Uhlmann','Unzicker','Ushenina',
'VachierLagrave','Vaganian','VallejoPons','VanWely','Vitiugov','Volokitin','Waitzkin','Wang','WangH','Wei','Winawer','Wojtaszek','Wojtkiewicz',
'Wolff','Xie','Xu','Ye','Yermolinsky','Yu','Yudasin','Zhu','Zukertort','Zvjaginsev']

In [165]:
def load_player_moves(player):
    filename_location = '../../app/data/json/as_color/{}.asblack_stats.json'.format(player)
    with open(filename_location) as data_file:    
        data = json.load(data_file)
        moves = data['moves']
        moves_df = pd.DataFrame(moves)
        moves_df = moves_df.fillna(0)
    return moves_df

In [199]:
def average_by_col(df):
    df = (df/df.sum())*100
    return df

In [192]:
# Setup Dataframe
combined_moves = pd.DataFrame(load_player_moves('Adams'))

for player in all_gms:
    moves = load_player_moves(player)
    combined_moves = pd.concat([combined_moves,moves]).groupby(level=0).sum()
    #print(player)
    #print(combined_moves.head(1))

In [193]:
combined_moves.head()

,Bc1,Bc8,Bf1,Bf8,Ke1,Ke8,Nb1,Nb8,Ng1,Ng8,...,d2,d7,e2,e7,f2,f7,g2,g7,h2,h7
a1-a2,0.0,0.0,0.0,0.0,647.0,31.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,2.0,2.0,0.0,1.0
a1-a3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
a1-a4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
a1-a5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0
a1-a6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0


In [194]:
combined_moves_average = (combined_moves/combined_moves.sum())*100

In [203]:
adams_diff_to_all = average_by_col(load_player_moves('Adams')) - combined_moves_average

In [204]:
adams_diff_to_all.to_json('adams_diff_to_all.json')

In [196]:
combined_moves_average.to_json('combined_everyones_moves_average.json')

In [ ]:
for player in all_gms:
    load_player_moves(player)